In [1]:
###cnn_ 얼굴인식

KeyboardInterrupt: 

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# 데이터 경로 설정
# 훈련 데이터와 검증 데이터의 경로를 설정합니다. 데이터는 각 클래스(사람)별로 별도의 디렉토리에 저장해야 한다.
train_dir = 'path/to/train_data'  # 훈련 데이터 경로
validation_dir = 'path/to/validation_data'  # 검증 데이터 경로


# Keras의 ImageDataGenerator를 사용하여 데이터 증강(augmentation) 및 전처리를 수행한다. 이는 모델의 일반화 성능을 향상시키는 데 도움을 준다.
# 이미지 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 제너레이터 설정
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train_data'

In [4]:
model = Sequential([ # 여기서 전체적인 구조슬라이드에서 보았던 그 구조를 시퀸스 즉 순서도로 그려주는 거라 생각하면 된다.
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.summary()

NameError: name 'train_generator' is not defined

In [5]:
model.compile(
    optimizer='adam', # 로스함수를 어떻게 최적화 시킬 건지에 대한 방법이다. 즉 거칠게 얘기하면 학습의 가장 기본단계인 알고리즘을 결정하는 거다.
    loss='categorical_crossentropy', # 로스함수 즉 손실치를 측정해서 그 손실치를 줄여가는 방향으로 학습할 때 그 로스를 어떻게 정의하느냐다.
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30
)

NameError: name 'model' is not defined

In [6]:
# 학습 과정에서의 정확도와 손실 변화를 시각화하여 모델의 성능을 평가한다.
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined

In [7]:
model.save('face_classification_model.h5')

NameError: name 'model' is not defined

In [8]:
from tensorflow.keras.preprocessing import image
# 새롭게 만든 이 함수는 이미지를 전처리하고 모델을 사용하여 예측한 후, 가장 가능성이 높은 클래스를 반환한다.
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class[0]

# 예측 예시
img_path = 'path/to/test_image.jpg'
predicted_class = predict_image(img_path, model)
print(f'Predicted class: {predicted_class}')# 반환 결과 출력

NameError: name 'model' is not defined

In [9]:
###RNN 예시

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

# 문장 데이터
words = ["I", "work", "at", "google"]
word_index = {word: i for i, word in enumerate(words)}
index_word = {i: word for i, word in enumerate(words)}

# 시퀀스 데이터 생성
def create_sequences(words, word_index, seq_length):
    X = []
    y = []
    for i in range(len(words) - seq_length):
        X.append([word_index[word] for word in words[i:i+seq_length]])
        y.append(word_index[words[i+seq_length]])
    return np.array(X), np.array(y)

seq_length = 3
X, y = create_sequences(words, word_index, seq_length)

# 모델 입력을 위한 형태 변환 (samples, time steps)
X = X.reshape((X.shape[0], seq_length))
y = y.reshape((y.shape[0],))


In [11]:
vocab_size = len(words)
embed_dim = 10

# 모델 구성
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_length),
    SimpleRNN(50, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
# 모델 훈련
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 1.3895 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 13ms/step - loss: 1.3750 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 1.3639 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 15ms/step - loss: 1.3541 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 14ms/step - loss: 1.3443 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 14ms/step - loss: 1.3342 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 15ms/step - loss: 1.3233 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 14ms/step - loss: 1.3115 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 23ms/step - loss: 1.2995 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 16ms/step - loss: 1.2873 - accuracy: 1.0000
Epoch 11/200
1/1 

In [13]:
# 예측 수행
def predict_next_word(model, sequence, word_index, index_word):
    seq = np.array([word_index[word] for word in sequence])
    seq = seq.reshape((1, seq_length))
    prediction = model.predict(seq, verbose=0)
    predicted_index = np.argmax(prediction)
    return index_word[predicted_index]

# 예측 테스트
sequence = ["I", "work", "at"]
predicted_word = predict_next_word(model, sequence, word_index, index_word)
print(f'Next word after "{sequence}" is "{predicted_word}"')


Next word after "['I', 'work', 'at']" is "google"


In [14]:
###DNN 예시

In [ ]:
from tensorflow import keras

In [15]:
import numpy as np

In [ ]:
x = np.array([0,1,2,3,4], dtype='float64')

In [ ]:
y=x*2+1

In [ ]:
print(y)

In [ ]:
model = keras.models.Sequential()

In [ ]:
model.add(keras.layers.Dense(1, input_shape=(1,)))

In [ ]:
model.compile('SGD', 'mse')

In [ ]:
model.fit(x,y, epochs=200)

In [ ]:
model.predict(x)